In [709]:
import boto3
from file_extraction import *
from to_dataframe import *
import pandas as pd
import datetime, json

In [710]:
s3_client = boto3.client('s3')
files_dict = {'academy_csv': [], 'json': [], 'txt': [], 'csv': []}

In [711]:
extract_file_type(s3_client, 'Talent', files_dict, 'txt')
applicants = files_dict['txt']

txt_df = convert_all_to_df(applicants, 'txt')

In [712]:
extract_file_type(s3_client, 'Talent', files_dict, 'json')
applicants = files_dict['json']

json_df = convert_all_to_df(applicants, 'json')

KeyboardInterrupt: 

In [ ]:
# json_file = s3_client.get_object(Bucket='data-eng-228-final-project', Key='Talent/10401.json')
# json_df = json_to_pd(json_file)
# json_df


,name,date,strengths,weaknesses,self_development,geo_flex,financial_support_self,result,course_interest
0,Haslett Steet,07/08/2019,"[Problem Solving, Efficient, Patient]",[Undisciplined],Yes,Yes,Yes,Pass,Engineering


In [ ]:
extract_file_type(s3_client, 'Talent', files_dict, 'csv')
applicants = files_dict['csv']

csv_df = convert_all_to_df(applicants, 'csv')


In [ ]:
duplicated_names = csv_df[csv_df.duplicated(subset='name', keep=False)]
print(duplicated_names)

    id                name  gender         dob                     email  \
52  53  Nicolette Bonehill  Female  05/04/1995  nbonehill1g@hubpages.com   
8    9   Shurlocke Cringle    Male  16/04/1992      scringle8@uol.com.br   
15  16   Shurlocke Cringle    Male  27/11/1995                       NaN   
9   10  Nicolette Bonehill  Female  14/08/1999       neebee9@addthis.com   

          city                 address postcode      phone_number  \
52  Birmingham     952 Eastwood Center      B40  +44-825-679-5108   
8     Whitwell  3592 Cardinal Junction     DL10               NaN   
15      Pentre      92427 Thierer Road      SY4  +44 947 829 3817   
9     East End        589 Columbus Way     BH21  +44-364-359-2606   

                        uni degree  invited_date          month     invited_by  
52     University of Oxford    3rd           5.0  February 2019   Stacey Broad  
8     South Bank University    3rd          31.0   January 2019  Rupert Ripple  
15  University of Leicester   

In [ ]:
extract_file_type(s3_client, 'Academy', files_dict, 'csv')
applicants = files_dict['academy_csv']

ac_csv_df = convert_all_to_df(applicants, 'csv')
ac_csv_df

,name,trainer,Analytic_W1,Independent_W1,Determined_W1,Professional_W1,Studious_W1,Imaginative_W1,Analytic_W2,Independent_W2,...,Determined_W9,Professional_W9,Studious_W9,Imaginative_W9,Analytic_W10,Independent_W10,Determined_W10,Professional_W10,Studious_W10,Imaginative_W10
0,Quintus Penella,Gregor Gomez,1,2,2,1,2,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Simon Murrey,Gregor Gomez,6,1,1,2,4,2,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gustaf Lude,Gregor Gomez,6,4,1,1,2,3,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Yolanda Fosse,Gregor Gomez,2,1,2,3,3,3,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lynnett Swin,Gregor Gomez,2,2,4,5,1,2,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Moritz Mosedall,Mohammad Velazquez,1,1,5,1,2,6,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Chaim Inseal,Mohammad Velazquez,1,3,3,4,1,2,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Gertruda Syddie,Mohammad Velazquez,3,1,2,8,1,4,2.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Thom Derwin,Mohammad Velazquez,3,7,3,3,3,1,2.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
duplicated_names = ac_csv_df[ac_csv_df.duplicated(subset='name', keep=False)]
print(duplicated_names)

Empty DataFrame
Columns: [name, trainer, Analytic_W1, Independent_W1, Determined_W1, Professional_W1, Studious_W1, Imaginative_W1, Analytic_W2, Independent_W2, Determined_W2, Professional_W2, Studious_W2, Imaginative_W2, Analytic_W3, Independent_W3, Determined_W3, Professional_W3, Studious_W3, Imaginative_W3, Analytic_W4, Independent_W4, Determined_W4, Professional_W4, Studious_W4, Imaginative_W4, Analytic_W5, Independent_W5, Determined_W5, Professional_W5, Studious_W5, Imaginative_W5, Analytic_W6, Independent_W6, Determined_W6, Professional_W6, Studious_W6, Imaginative_W6, Analytic_W7, Independent_W7, Determined_W7, Professional_W7, Studious_W7, Imaginative_W7, Analytic_W8, Independent_W8, Determined_W8, Professional_W8, Studious_W8, Imaginative_W8, Analytic_W9, Independent_W9, Determined_W9, Professional_W9, Studious_W9, Imaginative_W9, Analytic_W10, Independent_W10, Determined_W10, Professional_W10, Studious_W10, Imaginative_W10]
Index: []

[0 rows x 62 columns]


In [ ]:
# Extract names and dates from the Academy .csv files describing name of the course and date
def course_names() -> dict:
    course_table = []

    # List the objects in the S3 bucket
    response = s3_client.list_objects(Bucket="data-eng-228-final-project", Prefix="Academy")

    # Iterate through the objects in the S3 bucket
    for obj in response['Contents']:
        file_key = obj['Key']
        
        # Extract the course name from the file key
        file_name = file_key.split('/')[-1].split('.')[0].split('_')[:2]
        course_name = file_name[0] + '_' + file_name[1]
        course_group = file_name[0]
        course_date = file_key.split('/')[-1].split('.')[0].split('_')[-1:]
        course_table.append([course_name, course_group, course_date[0]])
    

        course_df = pd.DataFrame(course_table, columns=['course_name', 'course_group', 'course_date'])
    
    return course_df

course_names()

,course_name,course_group,course_date
0,Business_20,Business,2019-02-11
1,Business_21,Business,2019-03-18
2,Business_22,Business,2019-04-15
3,Business_23,Business,2019-05-20
4,Business_24,Business,2019-07-15
5,Business_25,Business,2019-07-29
6,Business_26,Business,2019-08-12
7,Business_27,Business,2019-09-16
8,Business_28,Business,2019-10-21
9,Business_29,Business,2019-11-18


In [ ]:
def academy_table():
    # Initialize an empty list to store data
    course_table = []

    # List the objects in the S3 bucket
    response = s3_client.list_objects(Bucket="data-eng-228-final-project", Prefix="Academy")

    # Iterate through the objects in the S3 bucket
    for obj in response['Contents']:
        file_key = obj['Key']
        
        # Extract the course name from the file key
        file_name = file_key.split('/')[-1].split('.')[0].split('_')[:2]
        course_name = file_name[0] + '_' + file_name[1]
        course_group = file_name[0]
        course_date = file_key.split('/')[-1].split('.')[0].split('_')[-1:]
        
        # Read the CSV file from S3 and extract student IDs
        csv_object = s3_client.get_object(Bucket="data-eng-228-final-project", Key=file_key)
        df = pd.read_csv(csv_object['Body'])
        student_ids = df['name'].tolist()
        
        # Append the data to the list
        course_table.extend([(student_id, course_name, course_date[0]) for student_id in student_ids])

    # Create the DataFrame
    course_df = pd.DataFrame(course_table, columns=['name', 'course_name', 'course_date'])

    # Return the DataFrame
    return course_df

academy_table()


,name,course_name,course_date
0,Quintus Penella,Business_20,2019-02-11
1,Simon Murrey,Business_20,2019-02-11
2,Gustaf Lude,Business_20,2019-02-11
3,Yolanda Fosse,Business_20,2019-02-11
4,Lynnett Swin,Business_20,2019-02-11
...,...,...,...
392,Moritz Mosedall,Engineering_29,2019-12-30
393,Chaim Inseal,Engineering_29,2019-12-30
394,Gertruda Syddie,Engineering_29,2019-12-30
395,Thom Derwin,Engineering_29,2019-12-30


In [ ]:
academy_table()

,name,course_name,course_date
0,Quintus Penella,Business_20,2019-02-11
1,Simon Murrey,Business_20,2019-02-11
2,Gustaf Lude,Business_20,2019-02-11
3,Yolanda Fosse,Business_20,2019-02-11
4,Lynnett Swin,Business_20,2019-02-11
...,...,...,...
392,Moritz Mosedall,Engineering_29,2019-12-30
393,Chaim Inseal,Engineering_29,2019-12-30
394,Gertruda Syddie,Engineering_29,2019-12-30
395,Thom Derwin,Engineering_29,2019-12-30


In [ ]:
# Amy's code for courses table
def create_courses(dataframe):
    courses_df  = dataframe
    courses_df = pd.DataFrame().assign(course = courses_df['course_name'], date = courses_df['course_date'])
    courses_df['course_id'] = courses_df.groupby(['course']).ngroup()
    courses_df = courses_df[['course', 'course_id', 'date']].copy()
    courses_df.set_index('course_id', inplace=True)
    return courses_df



create_courses(course_names())

,course,date
course_id,,
0,Business_20,2019-02-11
1,Business_21,2019-03-18
2,Business_22,2019-04-15
3,Business_23,2019-05-20
4,Business_24,2019-07-15
5,Business_25,2019-07-29
6,Business_26,2019-08-12
7,Business_27,2019-09-16
8,Business_28,2019-10-21


In [ ]:
courses = create_courses(course_names())

def academy_table_df(courses_df, academy_table_df):
    academy_df = []

    for index, row in academy_table_df.iterrows():
        # Assigning variables to values in candidate table
        inv_date = row['date']
        interest = row['course_interest']
        # 'name' to be changed to 'name_iD' from the candidates table
        candidate_id = row['name']
        
        if row['result'] == 'Pass':
            # Setting conditions to choose the courses after candidates invitation date
            # Grouping by interest
            start_date_condition = courses_df['date'] > inv_date
            course_name_condition = courses_df['course'].str.split('_').str[0] == interest
            sorted_dates = courses_df[course_name_condition & start_date_condition]
            candidate_course = sorted_dates.sort_values('date', ascending=False).iloc[0, 0]
            academy_df.append([candidate_id, candidate_course])
            academy_df = pd.DataFrame(academy_df, columns=['name_id', 'course'])
            

    merged_df = pd.merge(courses_df, academy_df, on='course', how='left')
    # merged_df.drop('course', axis=1, inplace=True)
    # merged_df.rename(columns={'course_id': 'course'}, inplace=True)

    return merged_df


print(academy_table_df(courses, json_df))

    

            course        date        name_id
0      Business_20  2019-02-11            NaN
1      Business_21  2019-03-18            NaN
2      Business_22  2019-04-15            NaN
3      Business_23  2019-05-20            NaN
4      Business_24  2019-07-15            NaN
5      Business_25  2019-07-29            NaN
6      Business_26  2019-08-12            NaN
7      Business_27  2019-09-16            NaN
8      Business_28  2019-10-21            NaN
9      Business_29  2019-11-18            NaN
10     Business_30  2019-12-30            NaN
11         Data_28  2019-02-18            NaN
12         Data_29  2019-03-04            NaN
13         Data_30  2019-04-08            NaN
14         Data_31  2019-05-20            NaN
15         Data_32  2019-07-22            NaN
16         Data_33  2019-08-05            NaN
17         Data_34  2019-08-19            NaN
18         Data_35  2019-09-23            NaN
19         Data_36  2019-10-28            NaN
20         Data_37  2019-11-18    